# Reference Architecture

## 1. Load all relevant libraries

In [2]:
#!pip install python-dotenv

In [3]:
import os

import dotenv
dotenv.load_dotenv()

import sys
sys.path.append('..')
import temimbo

import importlib

importlib.reload(temimbo)

<module 'temimbo' from 'c:\\Users\\franz\\OneDrive\\Desktop\\BA\\BA\\LanguageTeacher\\notebooks\\..\\temimbo\\__init__.py'>

## 2. Enable OpenAI-API access

In [4]:
# If this fails, check your .env file for the correct api key
connector_openai = temimbo.ConnectorOpenAI(
    openai_key = os.getenv('OPENAI_KEY')
)
tg = temimbo.TaskGenerator(connector_llm = connector_openai)
evaluator = temimbo.AnswerEvaluator(connector_llm = connector_openai)


ui = temimbo.UserInterface()
db = temimbo.DatabaseClientLocalFile(connection_string='./database')
formater = temimbo.Formater()



## 3. Load the learner profile

In [5]:
profile = await db.load_profile(id='Klaus')
domain = await ui.choose_domain(domain='text')
task_type = await ui.choose_task_type(task_type='single_choice')
level, training_goals_subset = await tg.incorporate_profile_in_task(profile, domain)

## 4. Generate the prompt for task generation & generate the task

In [6]:
prompt = await tg.generate_prompt(
    level = level,
    training_goals_subset = training_goals_subset,
    domain = domain,
    task_type = task_type,
)
print(prompt)

Pretend you are a academic english teacher. Generate a task description, and the task according to the format of the examples. No other text or tasks.
Generate a single choice with given answer possibilities, only one of the answers is correct. Generate only one task with four answer possibilities. Generate it for the area of text teaching, targeting on Talk about powerful cars.
Adjust the task dificulty to language level B1 (Can understand the main points of clear standard input on familiar matters regularly encountered in work, school, leisure, etc. Can deal with most situations likely to arise whilst travelling in an area where the language is spoken.  Can produce simple connected text on topics which are familiar or of personal interest. Can describe experiences and events, dreams, hopes & ambitions and briefly give reasons and explanations for opinions and plans).

The following are 2 examples of how the task should be generated with output format and style:

 Select the correct a

In [7]:
raw_output = await tg.generate_task(prompt)
print(raw_output)

Select the correct answer:
A powerful car can ___________
a) go very fast
b) fly in the sky
c) cook dinner
d) swim in the ocean


## 5. Input the learner answer

In [8]:
formatted_output_task = await formater.output_task_formatting(raw_output)
user_answer = await ui.answer_task(formatted_output_task, answer="c")
formated_user_answer = await formater.learner_answer_formatting(user_answer)


## 6. Generate binary evaluation, keywords & verbal feedback

In [9]:
NL_feedback, correctness, training_goals = await evaluator.evaluate_learner_answer(
    domain = domain,
    formatted_output_task = formatted_output_task,
    formated_user_answer = formated_user_answer,
    task_type = task_type,
)
print('NL_feedback =', NL_feedback)
print('correctness =', correctness)
print('training_goals =', training_goals)

NL_feedback = Thank you for your answer. However, the correct answer is (a) go very fast. A powerful car has the capability to go at high speeds. It seems like you need to practice more on vocabulary and comprehension. Keep up the good work and continue practicing to improve your skills!
correctness = False
training_goals = vocabulary_goals=['dinner'] grammar_goals=['verb form (can)'] text_goals=['comprehension']


## 7. Update the user profile

In [10]:
new_profile = await evaluator.update_learner_profile(training_goals,domain, profile, correctness)
print(f'Old profile:\n{profile}\n\nNew profile:\n{new_profile}')

Old profile:
name='Klaus' level=Level(vocabulary_level=4.1, grammar_level=4.0, text_level=2.1) training_goals=TrainingGoals(vocabulary_goals=['Cars', 'Mars'], grammar_goals=[], text_goals=['Talk about powerful cars'])

New profile:
name='Klaus' level=Level(vocabulary_level=4.1, grammar_level=4.0, text_level=2.0) training_goals=TrainingGoals(vocabulary_goals=['Mars', 'dinner', 'Cars'], grammar_goals=['verb form (can)'], text_goals=['comprehension', 'Talk about powerful cars'])
